## **Environment Setup**
Run all when initiating session

In [ ]:
# retrieve config file, if using colab (nothing should happen if you use Windows)
!cp './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/config.py' '.'
!mkdir data
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/data/' '.'
!mkdir models
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/models/' '.'
!mkdir technical
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/technical/' '.'
!mkdir utils
!cp -a './drive/My Drive/Live Workspace/generative-facial-cosmetics/encoder_engineering/utils/' '.'

In [ ]:
from config import *                            # config.py
from matplotlib import pyplot as plt 
import os
%matplotlib inline

## **GCS Integration**
Run all when initiating session

In [ ]:
if isWindows():
  import utils.gcs_windows as gcs 
elif isColab():
  import utils.gcs_colab as gcs
else:
  raise NotImplementedError('OS is not supported yet')

if isColab():
  gcs.init()
# no need of buckets for this project

## **Model Environment Setup**

In [ ]:
### tf-side debug ###
# more info: https://github.com/tensorflow/tensorflow/issues/29931
import tensorflow as tf
temp = tf.zeros([2, 16, 16, 3])  # Or tf.zeros
tf.keras.applications.vgg16.preprocess_input(temp)
print('')

In [ ]:
from data.pipeline import *
from technical.accelerators import strategy
from utils.generator_loading_utils import load_generator_checkpoint
from utils.encoder_loading_utils import load_encoder_checkpoint
from utils.face_utils import detect_and_crop_lips, replace_lips

In [ ]:
##### Create and load saved generator here (checkpoint) #####
# please put generator's saved checkpoint in OUTPUT/generator_checkpoints (GCS for colab)
RUN = True
if RUN:
  generator = load_generator_checkpoint(strategy, model_type='256')
  print(generator)
else:
  print("Running switch for this cell is off, skipping")

In [ ]:
##### Create and load saved encoder here (checkpoint) #####
# please put encoder's saved checkpoint in OUTPUT/encoder_checkpoints (GCS for colab)
RUN = True
if RUN:
  encoder = load_encoder_checkpoint(strategy)
  print(encoder)
else:
  print("Running switch for this cell is off, skipping")

In [ ]:
LATENT_SIZE = encoder.model.layers[-1].output.shape[1:] 
IMAGE_SHAPE = encoder.model.layers[0].output.shape[1:] 
IMAGE_SIZE = [IMAGE_SHAPE[0], IMAGE_SHAPE[0]]
print("Latent size: " + str(LATENT_SIZE))
print("Image shape: " + str(IMAGE_SHAPE))

## **Testing**

In [ ]:
import numpy as np
import os

In [ ]:
## input
img_name = 'test_2.png'
img_path = os.path.join(DIR, os.path.join('samples', img_name))
img = load_image(img_path)
#plt.imshow(img[:, :, [2, 1, 0]])

In [ ]:
## crop
cropped_lips, p_data = detect_and_crop_lips(img_full=img)
cropped_lips = np.array(cropped_lips)/255
#plt.imshow(cropped_lips)

In [ ]:
resized = cropped_lips #cv2.resize(cropped_lips, dsize=(90, 90), interpolation=cv2.INTER_CUBIC)
plt.imshow(resized)

In [ ]:
## Encode the cropped lips
encoded = encoder.model(np.expand_dims(resized, 0))
print(encoded)

In [ ]:
## Regeneration (w/o injections)
regenerated = generator.model(encoded, training=False)[0]
plt.imshow(regenerated)
print("Visual loss: " + str(float(tf.keras.losses.MSE(tf.keras.backend.flatten(resized), tf.keras.backend.flatten(regenerated)))))

In [ ]:
## Replacement
new_img = replace_lips(regenerated.numpy(), p_data, img_full=cv2.resize(img, dsize=(90, 90), interpolation=cv2.INTER_CUBIC))

## **Experiments**
Just a playground for trying out codes, nothing related at all, do not execute

In [ ]:
img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGBA))
plt.imshow(img_pil)

In [ ]:
min_x, min_y, max_x, max_y, offset_x, offset_y = p_data
cropped_lips_pil = Image.fromarray((cropped_lips*255).astype('uint8'))
cropped_lips_pil = cropped_lips_pil.crop((offset_x, offset_y, offset_x+(max_x-min_x), offset_y+(max_y-min_y)))
img_pil = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGBA))
img_pil.paste(cropped_lips_pil, (min_x, min_y), cropped_lips_pil)
plt.imshow(img_pil)

In [ ]:
min_x, min_y, max_x, max_y, offset_x, offset_y = p_data
img_tmp = (cropped_lips*255).astype('uint8')
cropped_lips_pil = Image.fromarray(img_tmp)
cropped_lips_pil = cropped_lips_pil.crop((offset_x, offset_y, offset_x+(max_x-min_x), offset_y+(max_y-min_y)))
plt.imshow(cropped_lips_pil)